In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest

In [ ]:

# Read log file
log_file_path = "/content/system_logs.txt"  # Update with your file path if needed
with open(log_file_path, "r") as file:
    logs = file.readlines()

# Parse logs into a structured DataFrame
data = []
for log in logs:
    parts = log.strip().split(" ", 3)  # Ensure the message part is captured fully
    if len(parts) < 4:
        continue  # Skip malformed lines
    timestamp = parts[0] + " " + parts[1]
    level = parts[2]
    message = parts[3]
    data.append([timestamp, level, message])

df = pd.DataFrame(data, columns=["timestamp", "level", "message"])

# Convert timestamp to datetime format for sorting
df["timestamp"] = pd.to_datetime(df["timestamp"])

# Assign numeric scores to log levels
level_mapping = {"INFO": 1, "WARNING": 2, "ERROR": 3, "CRITICAL": 4}
df["level_score"] = df["level"].map(level_mapping)

# Add message length as a new feature
df["message_length"] = df["message"].apply(len)



In [ ]:
# AI Model for Anomaly Detection (Isolation Forest)
model = IsolationForest(contamination=0.1, random_state=42)  # Lower contamination for better accuracy
df["anomaly"] = model.fit_predict(df[["level_score", "message_length"]])

# Mark anomalies in a readable format
df["is_anomaly"] = df["anomaly"].apply(lambda x: "❌ Anomaly" if x == -1 else "✅ Normal")

# Print only detected anomalies
anomalies = df[df["is_anomaly"] == "❌ Anomaly"]
print("\n🔍 **Detected Anomalies:**\n", anomalies)




🔍 **Detected Anomalies:**
               timestamp     level  \
19  2025-03-27 10:00:38   WARNING   
21  2025-03-27 10:01:45  CRITICAL   
23  2025-03-27 10:01:09  CRITICAL   
24  2025-03-27 10:02:00   WARNING   
26  2025-03-27 10:01:44     ERROR   
..                  ...       ...   
887 2025-03-27 10:14:47   WARNING   
941 2025-03-27 11:02:44   WARNING   
942 2025-03-27 10:47:06   WARNING   
955 2025-03-27 10:47:45  CRITICAL   
997 2025-03-27 11:06:28  CRITICAL   

                                            message  level_score  \
19                                   User logged in            2   
21                            User changed password            4   
23             Transaction rollback due to deadlock            4   
24        Slow query detected: SELECT * FROM orders            2   
26                                 CPU usage at 95%            3   
..                                              ...          ...   
887       Slow query detected: SELECT * FROM orders